In [1]:
%pip install --upgrade transformers accelerate pandas sentencepiece einops --quiet
%pip install bitsandbytes --quiet
%pip install sentence-transformers rank_bm25 --quiet
import json
import torch
import os
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ADITYA\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\ADITYA\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\Users\ADITYA\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\Users\ADITYA\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


Note: you may need to restart the kernel to use updated packages.


c:\Users\ADITYA\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
knowledge_base = [
    # ============================================
    # ALGEBRA - EASY LEVEL (alg_1 to alg_125)
    # ============================================
    {"questionID": "alg_1", "subject": "algebra", "question": "What is 2 + 2?", "answer": "4"},
    {"questionID": "alg_2", "subject": "algebra", "question": "What is 5 + 3?", "answer": "8"},
    {"questionID": "alg_3", "subject": "algebra", "question": "What is 10 - 4?", "answer": "6"},
    {"questionID": "alg_4", "subject": "algebra", "question": "What is 7 - 3?", "answer": "4"},
    {"questionID": "alg_5", "subject": "algebra", "question": "What is 3 × 4?", "answer": "12"},
    {"questionID": "alg_6", "subject": "algebra", "question": "What is 5 × 2?", "answer": "10"},
    {"questionID": "alg_7", "subject": "algebra", "question": "What is 10 ÷ 2?", "answer": "5"},
    {"questionID": "alg_8", "subject": "algebra", "question": "What is 15 ÷ 3?", "answer": "5"},
    {"questionID": "alg_9", "subject": "algebra", "question": "What is 1 + 1?", "answer": "2"},
    {"questionID": "alg_10", "subject": "algebra", "question": "What is 6 + 4?", "answer": "10"},
    {"questionID": "alg_11", "subject": "algebra", "question": "What is 8 - 5?", "answer": "3"},
    {"questionID": "alg_12", "subject": "algebra", "question": "What is 9 - 2?", "answer": "7"},
    {"questionID": "alg_13", "subject": "algebra", "question": "What is 2 × 5?", "answer": "10"},
    {"questionID": "alg_14", "subject": "algebra", "question": "What is 4 × 3?", "answer": "12"},
    {"questionID": "alg_15", "subject": "algebra", "question": "What is 20 ÷ 4?", "answer": "5"},
    {"questionID": "alg_16", "subject": "algebra", "question": "What is 18 ÷ 6?", "answer": "3"},
    {"questionID": "alg_17", "subject": "algebra", "question": "What is 3 + 7?", "answer": "10"},
    {"questionID": "alg_18", "subject": "algebra", "question": "What is 12 - 5?", "answer": "7"},
    {"questionID": "alg_19", "subject": "algebra", "question": "What is 6 × 2?", "answer": "12"},
    {"questionID": "alg_20", "subject": "algebra", "question": "What is 16 ÷ 4?", "answer": "4"},
    {"questionID": "alg_21", "subject": "algebra", "question": "What is 0 + 5?", "answer": "5"},
    {"questionID": "alg_22", "subject": "algebra", "question": "What is 10 - 0?", "answer": "10"},
    {"questionID": "alg_23", "subject": "algebra", "question": "What is 7 × 1?", "answer": "7"},
    {"questionID": "alg_24", "subject": "algebra", "question": "What is 8 ÷ 1?", "answer": "8"},
    {"questionID": "alg_25", "subject": "algebra", "question": "What is 4 + 6?", "answer": "10"},
    {"questionID": "alg_26", "subject": "algebra", "question": "What is 11 - 3?", "answer": "8"},
    {"questionID": "alg_27", "subject": "algebra", "question": "What is 5 × 5?", "answer": "25"},
    {"questionID": "alg_28", "subject": "algebra", "question": "What is 24 ÷ 6?", "answer": "4"},
    {"questionID": "alg_29", "subject": "algebra", "question": "What is 9 + 1?", "answer": "10"},
    {"questionID": "alg_30", "subject": "algebra", "question": "What is 15 - 7?", "answer": "8"},
    {"questionID": "alg_31", "subject": "algebra", "question": "What is 3 × 3?", "answer": "9"},
    {"questionID": "alg_32", "subject": "algebra", "question": "What is 14 ÷ 7?", "answer": "2"},
    {"questionID": "alg_33", "subject": "algebra", "question": "What is 2 + 8?", "answer": "10"},
    {"questionID": "alg_34", "subject": "algebra", "question": "What is 13 - 4?", "answer": "9"},
    {"questionID": "alg_35", "subject": "algebra", "question": "What is 7 × 2?", "answer": "14"},
    {"questionID": "alg_36", "subject": "algebra", "question": "What is 21 ÷ 3?", "answer": "7"},
    {"questionID": "alg_37", "subject": "algebra", "question": "What is 5 + 5?", "answer": "10"},
    {"questionID": "alg_38", "subject": "algebra", "question": "What is 20 - 8?", "answer": "12"},
    {"questionID": "alg_39", "subject": "algebra", "question": "What is 8 × 2?", "answer": "16"},
    {"questionID": "alg_40", "subject": "algebra", "question": "What is 30 ÷ 5?", "answer": "6"},
    {"questionID": "alg_41", "subject": "algebra", "question": "What is 6 + 6?", "answer": "12"},
    {"questionID": "alg_42", "subject": "algebra", "question": "What is 17 - 9?", "answer": "8"},
    {"questionID": "alg_43", "subject": "algebra", "question": "What is 9 × 2?", "answer": "18"},
    {"questionID": "alg_44", "subject": "algebra", "question": "What is 25 ÷ 5?", "answer": "5"},
    {"questionID": "alg_45", "subject": "algebra", "question": "What is 7 + 8?", "answer": "15"},
    {"questionID": "alg_46", "subject": "algebra", "question": "What is 19 - 6?", "answer": "13"},
    {"questionID": "alg_47", "subject": "algebra", "question": "What is 10 × 2?", "answer": "20"},
    {"questionID": "alg_48", "subject": "algebra", "question": "What is 36 ÷ 6?", "answer": "6"},
    {"questionID": "alg_49", "subject": "algebra", "question": "What is 11 + 9?", "answer": "20"},
    {"questionID": "alg_50", "subject": "algebra", "question": "What is 22 - 10?", "answer": "12"},
    {"questionID": "alg_51", "subject": "algebra", "question": "What is 6 × 6?", "answer": "36"},
    {"questionID": "alg_52", "subject": "algebra", "question": "What is 40 ÷ 8?", "answer": "5"},
    {"questionID": "alg_53", "subject": "algebra", "question": "What is 12 + 8?", "answer": "20"},
    {"questionID": "alg_54", "subject": "algebra", "question": "What is 25 - 15?", "answer": "10"},
    {"questionID": "alg_55", "subject": "algebra", "question": "What is 7 × 3?", "answer": "21"},
    {"questionID": "alg_56", "subject": "algebra", "question": "What is 45 ÷ 9?", "answer": "5"},
    {"questionID": "alg_57", "subject": "algebra", "question": "What is 14 + 6?", "answer": "20"},
    {"questionID": "alg_58", "subject": "algebra", "question": "What is 30 - 12?", "answer": "18"},
    {"questionID": "alg_59", "subject": "algebra", "question": "What is 8 × 3?", "answer": "24"},
    {"questionID": "alg_60", "subject": "algebra", "question": "What is 50 ÷ 10?", "answer": "5"},
    {"questionID": "alg_61", "subject": "algebra", "question": "What is 15 + 5?", "answer": "20"},
    {"questionID": "alg_62", "subject": "algebra", "question": "What is 28 - 8?", "answer": "20"},
    {"questionID": "alg_63", "subject": "algebra", "question": "What is 9 × 3?", "answer": "27"},
    {"questionID": "alg_64", "subject": "algebra", "question": "What is 32 ÷ 8?", "answer": "4"},
    {"questionID": "alg_65", "subject": "algebra", "question": "What is 13 + 7?", "answer": "20"},
    {"questionID": "alg_66", "subject": "algebra", "question": "What is 35 - 15?", "answer": "20"},
    {"questionID": "alg_67", "subject": "algebra", "question": "What is 4 × 5?", "answer": "20"},
    {"questionID": "alg_68", "subject": "algebra", "question": "What is 27 ÷ 3?", "answer": "9"},
    {"questionID": "alg_69", "subject": "algebra", "question": "What is 16 + 4?", "answer": "20"},
    {"questionID": "alg_70", "subject": "algebra", "question": "What is 40 - 20?", "answer": "20"},
    {"questionID": "alg_71", "subject": "algebra", "question": "What is 10 × 3?", "answer": "30"},
    {"questionID": "alg_72", "subject": "algebra", "question": "What is 48 ÷ 6?", "answer": "8"},
    {"questionID": "alg_73", "subject": "algebra", "question": "What is 18 + 2?", "answer": "20"},
    {"questionID": "alg_74", "subject": "algebra", "question": "What is 50 - 25?", "answer": "25"},
    {"questionID": "alg_75", "subject": "algebra", "question": "What is 5 × 6?", "answer": "30"},
    {"questionID": "alg_76", "subject": "algebra", "question": "What is 54 ÷ 9?", "answer": "6"},
    {"questionID": "alg_77", "subject": "algebra", "question": "What is 19 + 1?", "answer": "20"},
    {"questionID": "alg_78", "subject": "algebra", "question": "What is 45 - 20?", "answer": "25"},
    {"questionID": "alg_79", "subject": "algebra", "question": "What is 6 × 5?", "answer": "30"},
    {"questionID": "alg_80", "subject": "algebra", "question": "What is 63 ÷ 7?", "answer": "9"},
    {"questionID": "alg_81", "subject": "algebra", "question": "What is 17 + 3?", "answer": "20"},
    {"questionID": "alg_82", "subject": "algebra", "question": "What is 60 - 30?", "answer": "30"},
    {"questionID": "alg_83", "subject": "algebra", "question": "What is 7 × 5?", "answer": "35"},
    {"questionID": "alg_84", "subject": "algebra", "question": "What is 56 ÷ 8?", "answer": "7"},
    {"questionID": "alg_85", "subject": "algebra", "question": "What is 23 + 7?", "answer": "30"},
    {"questionID": "alg_86", "subject": "algebra", "question": "What is 55 - 25?", "answer": "30"},
    {"questionID": "alg_87", "subject": "algebra", "question": "What is 8 × 4?", "answer": "32"},
    {"questionID": "alg_88", "subject": "algebra", "question": "What is 72 ÷ 9?", "answer": "8"},
    {"questionID": "alg_89", "subject": "algebra", "question": "What is 21 + 9?", "answer": "30"},
    {"questionID": "alg_90", "subject": "algebra", "question": "What is 70 - 40?", "answer": "30"},
    {"questionID": "alg_91", "subject": "algebra", "question": "What is 9 × 4?", "answer": "36"},
    {"questionID": "alg_92", "subject": "algebra", "question": "What is 64 ÷ 8?", "answer": "8"},
    {"questionID": "alg_93", "subject": "algebra", "question": "What is 25 + 5?", "answer": "30"},
    {"questionID": "alg_94", "subject": "algebra", "question": "What is 80 - 50?", "answer": "30"},
    {"questionID": "alg_95", "subject": "algebra", "question": "What is 10 × 4?", "answer": "40"},
    {"questionID": "alg_96", "subject": "algebra", "question": "What is 81 ÷ 9?", "answer": "9"},
    {"questionID": "alg_97", "subject": "algebra", "question": "What is 26 + 4?", "answer": "30"},
    {"questionID": "alg_98", "subject": "algebra", "question": "What is 90 - 60?", "answer": "30"},
    {"questionID": "alg_99", "subject": "algebra", "question": "What is 5 × 8?", "answer": "40"},
    {"questionID": "alg_100", "subject": "algebra", "question": "What is 100 ÷ 10?", "answer": "10"},
    {"questionID": "alg_101", "subject": "algebra", "question": "What is 22 + 8?", "answer": "30"},
    {"questionID": "alg_102", "subject": "algebra", "question": "What is 100 - 50?", "answer": "50"},
    {"questionID": "alg_103", "subject": "algebra", "question": "What is 6 × 7?", "answer": "42"},
    {"questionID": "alg_104", "subject": "algebra", "question": "What is 35 ÷ 7?", "answer": "5"},
    {"questionID": "alg_105", "subject": "algebra", "question": "What is 24 + 16?", "answer": "40"},
    {"questionID": "alg_106", "subject": "algebra", "question": "What is 75 - 25?", "answer": "50"},
    {"questionID": "alg_107", "subject": "algebra", "question": "What is 7 × 7?", "answer": "49"},
    {"questionID": "alg_108", "subject": "algebra", "question": "What is 42 ÷ 6?", "answer": "7"},
    {"questionID": "alg_109", "subject": "algebra", "question": "What is 28 + 12?", "answer": "40"},
    {"questionID": "alg_110", "subject": "algebra", "question": "What is 85 - 35?", "answer": "50"},
    {"questionID": "alg_111", "subject": "algebra", "question": "What is 8 × 5?", "answer": "40"},
    {"questionID": "alg_112", "subject": "algebra", "question": "What is 49 ÷ 7?", "answer": "7"},
    {"questionID": "alg_113", "subject": "algebra", "question": "What is 30 + 10?", "answer": "40"},
    {"questionID": "alg_114", "subject": "algebra", "question": "What is 95 - 45?", "answer": "50"},
    {"questionID": "alg_115", "subject": "algebra", "question": "What is 9 × 5?", "answer": "45"},
    {"questionID": "alg_116", "subject": "algebra", "question": "What is 28 ÷ 4?", "answer": "7"},
    {"questionID": "alg_117", "subject": "algebra", "question": "What is 32 + 8?", "answer": "40"},
    {"questionID": "alg_118", "subject": "algebra", "question": "What is 100 - 60?", "answer": "40"},
    {"questionID": "alg_119", "subject": "algebra", "question": "What is 10 × 5?", "answer": "50"},
    {"questionID": "alg_120", "subject": "algebra", "question": "What is 100 ÷ 5?", "answer": "20"},
    {"questionID": "alg_121", "subject": "algebra", "question": "What is 35 + 15?", "answer": "50"},
    {"questionID": "alg_122", "subject": "algebra", "question": "What is 120 - 70?", "answer": "50"},
    {"questionID": "alg_123", "subject": "algebra", "question": "What is 12 × 3?", "answer": "36"},
    {"questionID": "alg_124", "subject": "algebra", "question": "What is 60 ÷ 12?", "answer": "5"},
    {"questionID": "alg_125", "subject": "algebra", "question": "What is 40 + 10?", "answer": "50"},

    # ============================================
    # GEOGRAPHY - EASY LEVEL (geo_1 to geo_125)
    # ============================================
    {"questionID":"geo_1","subject":"geography","question":"Capital of France?","answer":"Paris"},
    {"questionID":"geo_2","subject":"geography","question":"Capital of Italy?","answer":"Rome"},
    {"questionID":"geo_3","subject":"geography","question":"Capital of Spain?","answer":"Madrid"},
    {"questionID":"geo_4","subject":"geography","question":"Capital of Germany?","answer":"Berlin"},
    {"questionID":"geo_5","subject":"geography","question":"Capital of England?","answer":"London"},
    {"questionID":"geo_6","subject":"geography","question":"Capital of USA?","answer":"Washington D.C."},
    {"questionID":"geo_7","subject":"geography","question":"Capital of Canada?","answer":"Ottawa"},
    {"questionID":"geo_8","subject":"geography","question":"Capital of Japan?","answer":"Tokyo"},
    {"questionID":"geo_9","subject":"geography","question":"Capital of China?","answer":"Beijing"},
    {"questionID":"geo_10","subject":"geography","question":"Capital of India?","answer":"New Delhi"},
    {"questionID":"geo_11","subject":"geography","question":"Capital of Russia?","answer":"Moscow"},
    {"questionID":"geo_12","subject":"geography","question":"Capital of Brazil?","answer":"Brasília"},
    {"questionID":"geo_13","subject":"geography","question":"Capital of Australia?","answer":"Canberra"},
    {"questionID":"geo_14","subject":"geography","question":"Capital of Egypt?","answer":"Cairo"},
    {"questionID":"geo_15","subject":"geography","question":"Capital of Mexico?","answer":"Mexico City"},
    {"questionID":"geo_16","subject":"geography","question":"Largest ocean?","answer":"Pacific Ocean"},
    {"questionID":"geo_17","subject":"geography","question":"Smallest ocean?","answer":"Arctic Ocean"},
    {"questionID":"geo_18","subject":"geography","question":"Largest continent?","answer":"Asia"},
    {"questionID":"geo_19","subject":"geography","question":"Smallest continent?","answer":"Australia"},
    {"questionID":"geo_20","subject":"geography","question":"Longest river?","answer":"Nile"},
    {"questionID":"geo_21","subject":"geography","question":"Highest mountain?","answer":"Mount Everest"},
    {"questionID":"geo_22","subject":"geography","question":"Largest country?","answer":"Russia"},
    {"questionID":"geo_23","subject":"geography","question":"Smallest country?","answer":"Vatican City"},
    {"questionID":"geo_24","subject":"geography","question":"Largest island?","answer":"Greenland"},
    {"questionID":"geo_25","subject":"geography","question":"Largest desert?","answer":"Sahara"},
    {"questionID":"geo_26","subject":"geography","question":"How many continents?","answer":"Seven"},
    {"questionID":"geo_27","subject":"geography","question":"How many oceans?","answer":"Five"},
    {"questionID":"geo_28","subject":"geography","question":"Capital of Greece?","answer":"Athens"},
    {"questionID":"geo_29","subject":"geography","question":"Capital of Portugal?","answer":"Lisbon"},
    {"questionID":"geo_30","subject":"geography","question":"Capital of Turkey?","answer":"Ankara"},
    {"questionID":"geo_31","subject":"geography","question":"Capital of Thailand?","answer":"Bangkok"},
    {"questionID":"geo_32","subject":"geography","question":"Capital of Argentina?","answer":"Buenos Aires"},
    {"questionID":"geo_33","subject":"geography","question":"Capital of Poland?","answer":"Warsaw"},
    {"questionID":"geo_34","subject":"geography","question":"Capital of Sweden?","answer":"Stockholm"},
    {"questionID":"geo_35","subject":"geography","question":"Capital of Norway?","answer":"Oslo"},
    {"questionID":"geo_36","subject":"geography","question":"Capital of Denmark?","answer":"Copenhagen"},
    {"questionID":"geo_37","subject":"geography","question":"Capital of Finland?","answer":"Helsinki"},
    {"questionID":"geo_38","subject":"geography","question":"Capital of Austria?","answer":"Vienna"},
    {"questionID":"geo_39","subject":"geography","question":"Capital of Switzerland?","answer":"Bern"},
    {"questionID":"geo_40","subject":"geography","question":"Capital of Belgium?","answer":"Brussels"},
    {"questionID":"geo_41","subject":"geography","question":"Capital of Netherlands?","answer":"Amsterdam"},
    {"questionID":"geo_42","subject":"geography","question":"Capital of South Korea?","answer":"Seoul"},
    {"questionID":"geo_43","subject":"geography","question":"Capital of Indonesia?","answer":"Jakarta"},
    {"questionID":"geo_44","subject":"geography","question":"Capital of Saudi Arabia?","answer":"Riyadh"},
    {"questionID":"geo_45","subject":"geography","question":"Capital of New Zealand?","answer":"Wellington"},
    {"questionID":"geo_46","subject":"geography","question":"Capital of Ireland?","answer":"Dublin"},
    {"questionID":"geo_47","subject":"geography","question":"Capital of Scotland?","answer":"Edinburgh"},
    {"questionID":"geo_48","subject":"geography","question":"Capital of Pakistan?","answer":"Islamabad"},
    {"questionID":"geo_49","subject":"geography","question":"Capital of Bangladesh?","answer":"Dhaka"},
    {"questionID":"geo_50","subject":"geography","question":"Capital of Vietnam?","answer":"Hanoi"},
    {"questionID":"geo_51","subject":"geography","question":"Capital of Philippines?","answer":"Manila"},
    {"questionID":"geo_52","subject":"geography","question":"Capital of Malaysia?","answer":"Kuala Lumpur"},
    {"questionID":"geo_53","subject":"geography","question":"Capital of Singapore?","answer":"Singapore"},
    {"questionID":"geo_54","subject":"geography","question":"Capital of Iran?","answer":"Tehran"},
    {"questionID":"geo_55","subject":"geography","question":"Capital of Iraq?","answer":"Baghdad"},
    {"questionID":"geo_56","subject":"geography","question":"Capital of Israel?","answer":"Jerusalem"},
    {"questionID":"geo_57","subject":"geography","question":"Capital of Jordan?","answer":"Amman"},
    {"questionID":"geo_58","subject":"geography","question":"Capital of Lebanon?","answer":"Beirut"},
    {"questionID":"geo_59","subject":"geography","question":"Capital of Syria?","answer":"Damascus"},
    {"questionID":"geo_60","subject":"geography","question":"Capital of Nigeria?","answer":"Abuja"},
    {"questionID":"geo_61","subject":"geography","question":"Capital of Kenya?","answer":"Nairobi"},
    {"questionID":"geo_62","subject":"geography","question":"Capital of Ethiopia?","answer":"Addis Ababa"},
    {"questionID":"geo_63","subject":"geography","question":"Capital of Ghana?","answer":"Accra"},
    {"questionID":"geo_64","subject":"geography","question":"Capital of Morocco?","answer":"Rabat"},
    {"questionID":"geo_65","subject":"geography","question":"Capital of Algeria?","answer":"Algiers"},
    {"questionID":"geo_66","subject":"geography","question":"Capital of Tunisia?","answer":"Tunis"},
    {"questionID":"geo_67","subject":"geography","question":"Capital of Libya?","answer":"Tripoli"},
    {"questionID":"geo_68","subject":"geography","question":"Capital of Peru?","answer":"Lima"},
    {"questionID":"geo_69","subject":"geography","question":"Capital of Chile?","answer":"Santiago"},
    {"questionID":"geo_70","subject":"geography","question":"Capital of Colombia?","answer":"Bogotá"},
    {"questionID":"geo_71","subject":"geography","question":"Capital of Venezuela?","answer":"Caracas"},
    {"questionID":"geo_72","subject":"geography","question":"Capital of Ecuador?","answer":"Quito"},
    {"questionID":"geo_73","subject":"geography","question":"Capital of Cuba?","answer":"Havana"},
    {"questionID":"geo_74","subject":"geography","question":"Capital of Jamaica?","answer":"Kingston"},
    {"questionID":"geo_75","subject":"geography","question":"Capital of Ukraine?","answer":"Kyiv"},
    {"questionID":"geo_76","subject":"geography","question":"Capital of Romania?","answer":"Bucharest"},
    {"questionID":"geo_77","subject":"geography","question":"Capital of Hungary?","answer":"Budapest"},
    {"questionID":"geo_78","subject":"geography","question":"Capital of Czech Republic?","answer":"Prague"},
    {"questionID":"geo_79","subject":"geography","question":"Capital of Slovakia?","answer":"Bratislava"},
    {"questionID":"geo_80","subject":"geography","question":"Capital of Bulgaria?","answer":"Sofia"},
    {"questionID":"geo_81","subject":"geography","question":"Capital of Serbia?","answer":"Belgrade"},
    {"questionID":"geo_82","subject":"geography","question":"Capital of Croatia?","answer":"Zagreb"},
    {"questionID":"geo_83","subject":"geography","question":"Capital of Iceland?","answer":"Reykjavik"},
    {"questionID":"geo_84","subject":"geography","question":"Where is Eiffel Tower?","answer":"Paris"},
    {"questionID":"geo_85","subject":"geography","question":"Where is Big Ben?","answer":"London"},
    {"questionID":"geo_86","subject":"geography","question":"Where is Statue of Liberty?","answer":"New York"},
    {"questionID":"geo_87","subject":"geography","question":"Where is Taj Mahal?","answer":"India"},
    {"questionID":"geo_88","subject":"geography","question":"Where is Great Wall?","answer":"China"},
    {"questionID":"geo_89","subject":"geography","question":"Where is Pyramids?","answer":"Egypt"},
    {"questionID":"geo_90","subject":"geography","question":"Where is Colosseum?","answer":"Rome"},
    {"questionID":"geo_91","subject":"geography","question":"Language of Brazil?","answer":"Portuguese"},
    {"questionID":"geo_92","subject":"geography","question":"Language of Spain?","answer":"Spanish"},
    {"questionID":"geo_93","subject":"geography","question":"Language of France?","answer":"French"},
    {"questionID":"geo_94","subject":"geography","question":"Language of Germany?","answer":"German"},
    {"questionID":"geo_95","subject":"geography","question":"Language of China?","answer":"Chinese"},
    {"questionID":"geo_96","subject":"geography","question":"Language of Japan?","answer":"Japanese"},
    {"questionID":"geo_97","subject":"geography","question":"Language of England?","answer":"English"},
    {"questionID":"geo_98","subject":"geography","question":"Second largest ocean?","answer":"Atlantic"},
    {"questionID":"geo_99","subject":"geography","question":"Third largest ocean?","answer":"Indian"},
    {"questionID":"geo_100","subject":"geography","question":"Second largest continent?","answer":"Africa"},
    {"questionID":"geo_101","subject":"geography","question":"Third largest continent?","answer":"North America"},
    {"questionID":"geo_102","subject":"geography","question":"Second longest river?","answer":"Amazon"},
    {"questionID":"geo_103","subject":"geography","question":"Second highest mountain?","answer":"K2"},
    {"questionID":"geo_104","subject":"geography","question":"Second largest country?","answer":"Canada"},
    {"questionID":"geo_105","subject":"geography","question":"Third largest country?","answer":"United States"},
    {"questionID":"geo_106","subject":"geography","question":"Where is Sahara?","answer":"Africa"},
    {"questionID":"geo_107","subject":"geography","question":"Where is Amazon Rainforest?","answer":"South America"},
    {"questionID":"geo_108","subject":"geography","question":"Where is Mount Everest?","answer":"Nepal"},
    {"questionID":"geo_109","subject":"geography","question":"Deepest ocean?","answer":"Pacific"},
    {"questionID":"geo_110","subject":"geography","question":"Hottest continent?","answer":"Africa"},
    {"questionID":"geo_111","subject":"geography","question":"Coldest continent?","answer":"Antarctica"},
    {"questionID":"geo_112","subject":"geography","question":"Capital of UAE?","answer":"Abu Dhabi"},
    {"questionID":"geo_113","subject":"geography","question":"Capital of Qatar?","answer":"Doha"},
    {"questionID":"geo_114","subject":"geography","question":"Capital of Kuwait?","answer":"Kuwait City"},
    {"questionID":"geo_115","subject":"geography","question":"Capital of Nepal?","answer":"Kathmandu"},
    {"questionID":"geo_116","subject":"geography","question":"Capital of Sri Lanka?","answer":"Colombo"},
    {"questionID":"geo_117","subject":"geography","question":"Capital of Myanmar?","answer":"Naypyidaw"},
    {"questionID":"geo_118","subject":"geography","question":"Capital of Afghanistan?","answer":"Kabul"},
    {"questionID":"geo_119","subject":"geography","question":"Capital of Mongolia?","answer":"Ulaanbaatar"},
    {"questionID":"geo_120","subject":"geography","question":"Capital of North Korea?","answer":"Pyongyang"},
    {"questionID":"geo_121","subject":"geography","question":"Capital of Taiwan?","answer":"Taipei"},
    {"questionID":"geo_122","subject":"geography","question":"Capital of Cambodia?","answer":"Phnom Penh"},
    {"questionID":"geo_123","subject":"geography","question":"Capital of Laos?","answer":"Vientiane"},
    {"questionID":"geo_124","subject":"geography","question":"Capital of Brunei?","answer":"Bandar Seri Begawan"},
    {"questionID":"geo_125","subject":"geography","question":"Capital of Maldives?","answer":"Malé"},

    # ============================================
    # HISTORY - EASY LEVEL (hist_1 to hist_125)
    # ============================================
    {"questionID":"hist_1","subject":"history","question":"First US President?","answer":"George Washington"},
    {"questionID":"hist_2","subject":"history","question":"Discovered America?","answer":"Christopher Columbus"},
    {"questionID":"hist_3","subject":"history","question":"Invented light bulb?","answer":"Thomas Edison"},
    {"questionID":"hist_4","subject":"history","question":"Invented telephone?","answer":"Alexander Graham Bell"},
    {"questionID":"hist_5","subject":"history","question":"First man on moon?","answer":"Neil Armstrong"},
    {"questionID":"hist_6","subject":"history","question":"Painted Mona Lisa?","answer":"Leonardo da Vinci"},
    {"questionID":"hist_7","subject":"history","question":"Wrote Romeo and Juliet?","answer":"Shakespeare"},
    {"questionID":"hist_8","subject":"history","question":"Egyptian queen?","answer":"Cleopatra"},
    {"questionID":"hist_9","subject":"history","question":"Roman emperor?","answer":"Julius Caesar"},
    {"questionID":"hist_10","subject":"history","question":"Greek philosopher?","answer":"Aristotle"},
    {"questionID":"hist_11","subject":"history","question":"Invented airplane?","answer":"Wright Brothers"},
    {"questionID":"hist_12","subject":"history","question":"Indian independence leader?","answer":"Mahatma Gandhi"},
    {"questionID":"hist_13","subject":"history","question":"South African leader?","answer":"Nelson Mandela"},
    {"questionID":"hist_14","subject":"history","question":"US civil rights leader?","answer":"Martin Luther King Jr."},
    {"questionID":"hist_15","subject":"history","question":"First emperor of China?","answer":"Qin Shi Huang"},
    {"questionID":"hist_16","subject":"history","question":"Napoleon's empire?","answer":"French Empire"},
    {"questionID":"hist_17","subject":"history","question":"British WWII leader?","answer":"Winston Churchill"},
    {"questionID":"hist_18","subject":"history","question":"US WWII President?","answer":"Franklin D. Roosevelt"},
    {"questionID":"hist_19","subject":"history","question":"Moon landing year?","answer":"1969"},
    {"questionID":"hist_20","subject":"history","question":"WWII ended year?","answer":"1945"},
    {"questionID":"hist_21","subject":"history","question":"WWI started year?","answer":"1914"},
    {"questionID":"hist_22","subject":"history","question":"Columbus voyage year?","answer":"1492"},
    {"questionID":"hist_23","subject":"history","question":"French Revolution year?","answer":"1789"},
    {"questionID":"hist_24","subject":"history","question":"Berlin Wall fell year?","answer":"1989"},
    {"questionID":"hist_25","subject":"history","question":"Soviet Union collapsed?","answer":"1991"},
    {"questionID":"hist_26","subject":"history","question":"Titanic sank year?","answer":"1912"},
    {"questionID":"hist_27","subject":"history","question":"First airplane flight?","answer":"1903"},
    {"questionID":"hist_28","subject":"history","question":"Who was Mozart?","answer":"Composer"},
    {"questionID":"hist_29","subject":"history","question":"Who was Beethoven?","answer":"Composer"},
    {"questionID":"hist_30","subject":"history","question":"Who was Picasso?","answer":"Painter"},
    {"questionID":"hist_31","subject":"history","question":"Who was Einstein?","answer":"Physicist"},
    {"questionID":"hist_32","subject":"history","question":"Who was Newton?","answer":"Scientist"},
    {"questionID":"hist_33","subject":"history","question":"Who was Darwin?","answer":"Naturalist"},
    {"questionID":"hist_34","subject":"history","question":"Who was Galileo?","answer":"Astronomer"},
    {"questionID":"hist_35","subject":"history","question":"Who was Confucius?","answer":"Philosopher"},
    {"questionID":"hist_36","subject":"history","question":"Who was Buddha?","answer":"Religious founder"},
    {"questionID":"hist_37","subject":"history","question":"Who built Taj Mahal?","answer":"Shah Jahan"},
    {"questionID":"hist_38","subject":"history","question":"Who was Alexander?","answer":"Macedonian king"},
    {"questionID":"hist_39","subject":"history","question":"Who was Genghis Khan?","answer":"Mongol emperor"},
    {"questionID":"hist_40","subject":"history","question":"Who was Henry VIII?","answer":"English king"},
    {"questionID":"hist_41","subject":"history","question":"Who was Elizabeth I?","answer":"English queen"},
    {"questionID":"hist_42","subject":"history","question":"Who was Queen Victoria?","answer":"British queen"},
    {"questionID":"hist_43","subject":"history","question":"Who was Joan of Arc?","answer":"French heroine"},
    {"questionID":"hist_44","subject":"history","question":"Second US President?","answer":"John Adams"},
    {"questionID":"hist_45","subject":"history","question":"Third US President?","answer":"Thomas Jefferson"},
    {"questionID":"hist_46","subject":"history","question":"16th US President?","answer":"Abraham Lincoln"},
    {"questionID":"hist_47","subject":"history","question":"Who was Marco Polo?","answer":"Explorer"},
    {"questionID":"hist_48","subject":"history","question":"Who was Magellan?","answer":"Explorer"},
    {"questionID":"hist_49","subject":"history","question":"Who discovered penicillin?","answer":"Alexander Fleming"},
    {"questionID":"hist_50","subject":"history","question":"Who painted Starry Night?","answer":"Van Gogh"},
    {"questionID":"hist_51","subject":"history","question":"Who was Socrates?","answer":"Philosopher"},
    {"questionID":"hist_52","subject":"history","question":"Who was Plato?","answer":"Philosopher"},
    {"questionID":"hist_53","subject":"history","question":"Who was Homer?","answer":"Poet"},
    {"questionID":"hist_54","subject":"history","question":"Who wrote Iliad?","answer":"Homer"},
    {"questionID":"hist_55","subject":"history","question":"Who was Michelangelo?","answer":"Artist"},
    {"questionID":"hist_56","subject":"history","question":"Built Great Wall?","answer":"Chinese dynasties"},
    {"questionID":"hist_57","subject":"history","question":"Built pyramids?","answer":"Ancient Egyptians"},
    {"questionID":"hist_58","subject":"history","question":"Roman Empire capital?","answer":"Rome"},
    {"questionID":"hist_59","subject":"history","question":"Ancient Greece capital?","answer":"Athens"},
    {"questionID":"hist_60","subject":"history","question":"Who was Spartacus?","answer":"Gladiator"},
    {"questionID":"hist_61","subject":"history","question":"Who was Hannibal?","answer":"Military commander"},
    {"questionID":"hist_62","subject":"history","question":"Who was Charlemagne?","answer":"Holy Roman Emperor"},
    {"questionID":"hist_63","subject":"history","question":"Who was William Shakespeare?","answer":"Playwright"},
    {"questionID":"hist_64","subject":"history","question":"Who was Marie Curie?","answer":"Scientist"},
    {"questionID":"hist_65","subject":"history","question":"Pearl Harbor year?","answer":"1941"},
    {"questionID":"hist_66","subject":"history","question":"D-Day year?","answer":"1944"},
    {"questionID":"hist_67","subject":"history","question":"Hiroshima bombing year?","answer":"1945"},
    {"questionID":"hist_68","subject":"history","question":"United Nations founded?","answer":"1945"},
    {"questionID":"hist_69","subject":"history","question":"Berlin Wall built?","answer":"1961"},
    {"questionID":"hist_70","subject":"history","question":"First human in space?","answer":"Yuri Gagarin"},
    {"questionID":"hist_71","subject":"history","question":"Declaration of Independence?","answer":"1776"},
    {"questionID":"hist_72","subject":"history","question":"US Constitution ratified?","answer":"1788"},
    {"questionID":"hist_73","subject":"history","question":"Who was Benjamin Franklin?","answer":"Founding Father"},
    {"questionID":"hist_74","subject":"history","question":"American Civil War?","answer":"1861-1865"},
    {"questionID":"hist_75","subject":"history","question":"Who signed Emancipation?","answer":"Abraham Lincoln"},
    {"questionID":"hist_76","subject":"history","question":"Industrial Revolution where?","answer":"Great Britain"},
    {"questionID":"hist_77","subject":"history","question":"Printing press inventor?","answer":"Johannes Gutenberg"},
    {"questionID":"hist_78","subject":"history","question":"Protestant Reformation leader?","answer":"Martin Luther"},
    {"questionID":"hist_79","subject":"history","question":"Who was Cleopatra?","answer":"Egyptian queen"},
    {"questionID":"hist_80","subject":"history","question":"Roman Empire fell?","answer":"476 AD"},
    {"questionID":"hist_81","subject":"history","question":"Battle of Hastings?","answer":"1066"},
    {"questionID":"hist_82","subject":"history","question":"Magna Carta signed?","answer":"1215"},
    {"questionID":"hist_83","subject":"history","question":"Black Death period?","answer":"1347-1353"},
    {"questionID":"hist_84","subject":"history","question":"What was Renaissance?","answer":"Cultural rebirth"},
    {"questionID":"hist_85","subject":"history","question":"Spanish Armada?","answer":"1588"},
    {"questionID":"hist_86","subject":"history","question":"Russian Revolution?","answer":"1917"},
    {"questionID":"hist_87","subject":"history","question":"Who was Lenin?","answer":"Russian leader"},
    {"questionID":"hist_88","subject":"history","question":"Cold War period?","answer":"1947-1991"},
    {"questionID":"hist_89","subject":"history","question":"Great Depression started?","answer":"1929"},
    {"questionID":"hist_90","subject":"history","question":"Who was Ramses II?","answer":"Egyptian pharaoh"},
    {"questionID":"hist_91","subject":"history","question":"Who was King Tut?","answer":"Egyptian pharaoh"},
    {"questionID":"hist_92","subject":"history","question":"Battle of Waterloo?","answer":"1815"},
    {"questionID":"hist_93","subject":"history","question":"Where was Napoleon exiled?","answer":"Saint Helena"},
    {"questionID":"hist_94","subject":"history","question":"Byzantine Empire capital?","answer":"Constantinople"},
    {"questionID":"hist_95","subject":"history","question":"Ottoman Empire capital?","answer":"Istanbul"},
    {"questionID":"hist_96","subject":"history","question":"Who was Suleiman?","answer":"Ottoman sultan"},
    {"questionID":"hist_97","subject":"history","question":"Who was Akbar?","answer":"Mughal emperor"},
    {"questionID":"hist_98","subject":"history","question":"Crusades period?","answer":"1096-1291"},
    {"questionID":"hist_99","subject":"history","question":"What was Black Death?","answer":"Plague pandemic"},
    {"questionID":"hist_100","subject":"history","question":"Who was Attila?","answer":"Hun leader"},
    {"questionID":"hist_101","subject":"history","question":"Who was Hammurabi?","answer":"Babylonian king"},
    {"questionID":"hist_102","subject":"history","question":"Who was Pericles?","answer":"Athenian leader"},
    {"questionID":"hist_103","subject":"history","question":"Battle of Marathon?","answer":"490 BC"},
    {"questionID":"hist_104","subject":"history","question":"Mount Vesuvius eruption?","answer":"79 AD"},
    {"questionID":"hist_105","subject":"history","question":"Pompeii destroyed?","answer":"79 AD"},
    {"questionID":"hist_106","subject":"history","question":"Great Fire of London?","answer":"1666"},
    {"questionID":"hist_107","subject":"history","question":"Who was Copernicus?","answer":"Astronomer"},
    {"questionID":"hist_108","subject":"history","question":"American Revolution?","answer":"1775-1783"},
    {"questionID":"hist_109","subject":"history","question":"Who was Louis XIV?","answer":"French king"},
    {"questionID":"hist_110","subject":"history","question":"Who was Peter the Great?","answer":"Russian czar"},
    {"questionID":"hist_111","subject":"history","question":"Who was Catherine Great?","answer":"Russian empress"},
    {"questionID":"hist_112","subject":"history","question":"Hundred Years War?","answer":"1337-1453"},
    {"questionID":"hist_113","subject":"history","question":"Victorian era period?","answer":"1837-1901"},
    {"questionID":"hist_114","subject":"history","question":"Byzantine fell?","answer":"1453"},
    {"questionID":"hist_115","subject":"history","question":"Ottoman dissolved?","answer":"1922"},
    {"questionID":"hist_116","subject":"history","question":"Who was Saladin?","answer":"Muslim leader"},
    {"questionID":"hist_117","subject":"history","question":"King Tut tomb discovered?","answer":"1922"},
    {"questionID":"hist_118","subject":"history","question":"Who discovered Tut tomb?","answer":"Howard Carter"},
    {"questionID":"hist_119","subject":"history","question":"Egyptian writing system?","answer":"Hieroglyphics"},
    {"questionID":"hist_120","subject":"history","question":"Buddhism founder?","answer":"Siddhartha Gautama"},
    {"questionID":"hist_121","subject":"history","question":"Who wrote Communist?","answer":"Karl Marx"},
    {"questionID":"hist_122","subject":"history","question":"Manhattan Project purpose?","answer":"Atomic bomb"},
    {"questionID":"hist_123","subject":"history","question":"First UK Prime Minister?","answer":"Robert Walpole"},
    {"questionID":"hist_124","subject":"history","question":"First Indian president?","answer":"Rajendra Prasad"},
    {"questionID":"hist_125","subject":"history","question":"First woman Nobel?","answer":"Marie Curie"},

    # ============================================
    # CHINESE - EASY LEVEL (chin_1 to chin_125)
    # ============================================
    {"questionID":"chin_2","subject":"chinese","question":"一是什么意思？","answer":"一"},
    {"questionID":"chin_3","subject":"chinese","question":"二是什么意思？","answer":"二"},
    {"questionID":"chin_4","subject":"chinese","question":"三是什么意思？","answer":"三"},
    {"questionID":"chin_5","subject":"chinese","question":"四是什么意思？","answer":"四"},
    {"questionID":"chin_6","subject":"chinese","question":"五是什么意思？","answer":"五"},
    {"questionID":"chin_7","subject":"chinese","question":"六是什么意思？","answer":"六"},
    {"questionID":"chin_8","subject":"chinese","question":"七是什么意思？","answer":"七"},
    {"questionID":"chin_9","subject":"chinese","question":"八是什么意思？","answer":"八"},
    {"questionID":"chin_10","subject":"chinese","question":"九是什么意思？","answer":"九"},
    {"questionID":"chin_11","subject":"chinese","question":"十是什么意思？","answer":"十"},
    {"questionID":"chin_12","subject":"chinese","question":"你好是什么意思？","answer":"你好"},
    {"questionID":"chin_13","subject":"chinese","question":"谢谢是什么意思？","answer":"谢谢"},
    {"questionID":"chin_14","subject":"chinese","question":"再见是什么意思？","answer":"再见"},
    {"questionID":"chin_15","subject":"chinese","question":"对不起是什么意思？","answer":"对不起"},
    {"questionID":"chin_16","subject":"chinese","question":"请是什么意思？","answer":"请"},
    {"questionID":"chin_17","subject":"chinese","question":"是是什么意思？","answer":"是"},
    {"questionID":"chin_18","subject":"chinese","question":"不是什么意思？","answer":"不"},
    {"questionID":"chin_19","subject":"chinese","question":"好是什么意思？","answer":"好"},
    {"questionID":"chin_20","subject":"chinese","question":"坏是什么意思？","answer":"坏"},
    {"questionID":"chin_21","subject":"chinese","question":"大是什么意思？","answer":"大"},
    {"questionID":"chin_22","subject":"chinese","question":"小是什么意思？","answer":"小"},
    {"questionID":"chin_23","subject":"chinese","question":"爱是什么意思？","answer":"爱"},
    {"questionID":"chin_24","subject":"chinese","question":"水是什么意思？","answer":"水"},
    {"questionID":"chin_25","subject":"chinese","question":"火是什么意思？","answer":"火"},
    {"questionID":"chin_26","subject":"chinese","question":"人是什么意思？","answer":"人"},
    {"questionID":"chin_27","subject":"chinese","question":"男是什么意思？","answer":"男"},
    {"questionID":"chin_28","subject":"chinese","question":"女是什么意思？","answer":"女"},
    {"questionID":"chin_29","subject":"chinese","question":"家是什么意思？","answer":"家"},
    {"questionID":"chin_30","subject":"chinese","question":"书是什么意思？","answer":"书"},
    {"questionID":"chin_31","subject":"chinese","question":"学校是什么意思？","answer":"学校"},
    {"questionID":"chin_32","subject":"chinese","question":"老师是什么意思？","answer":"老师"},
    {"questionID":"chin_33","subject":"chinese","question":"学生是什么意思？","answer":"学生"},
    {"questionID":"chin_34","subject":"chinese","question":"朋友是什么意思？","answer":"朋友"},
    {"questionID":"chin_35","subject":"chinese","question":"父亲是什么意思？","answer":"父亲"},
    {"questionID":"chin_36","subject":"chinese","question":"母亲是什么意思？","answer":"母亲"},
    {"questionID":"chin_37","subject":"chinese","question":"儿子是什么意思？","answer":"儿子"},
    {"questionID":"chin_38","subject":"chinese","question":"女儿是什么意思？","answer":"女儿"},
    {"questionID":"chin_39","subject":"chinese","question":"哥哥是什么意思？","answer":"哥哥"},
    {"questionID":"chin_40","subject":"chinese","question":"姐姐是什么意思？","answer":"姐姐"},
    {"questionID":"chin_41","subject":"chinese","question":"吃是什么意思？","answer":"吃"},
    {"questionID":"chin_42","subject":"chinese","question":"喝是什么意思？","answer":"喝"},
    {"questionID":"chin_43","subject":"chinese","question":"睡觉是什么意思？","answer":"睡觉"},
    {"questionID":"chin_44","subject":"chinese","question":"工作是什么意思？","answer":"工作"},
    {"questionID":"chin_45","subject":"chinese","question":"钱是什么意思？","answer":"钱"},
    {"questionID":"chin_46","subject":"chinese","question":"时间是什么意思？","answer":"时间"},
    {"questionID":"chin_47","subject":"chinese","question":"今天是什么意思？","answer":"今天"},
    {"questionID":"chin_48","subject":"chinese","question":"明天是什么意思？","answer":"明天"},
    {"questionID":"chin_49","subject":"chinese","question":"昨天是什么意思？","answer":"昨天"},
    {"questionID":"chin_50","subject":"chinese","question":"早上是什么意思？","answer":"早上"},
    {"questionID":"chin_51","subject":"chinese","question":"晚上是什么意思？","answer":"晚上"},
    {"questionID":"chin_52","subject":"chinese","question":"中午是什么意思？","answer":"中午"},
    {"questionID":"chin_53","subject":"chinese","question":"年是什么意思？","answer":"年"},
    {"questionID":"chin_54","subject":"chinese","question":"月是什么意思？","answer":"月"},
    {"questionID":"chin_55","subject":"chinese","question":"日是什么意思？","answer":"日"},
    {"questionID":"chin_56","subject":"chinese","question":"星期是什么意思？","answer":"星期"},
    {"questionID":"chin_57","subject":"chinese","question":"快是什么意思？","answer":"快"},
    {"questionID":"chin_58","subject":"chinese","question":"慢是什么意思？","answer":"慢"},
    {"questionID":"chin_59","subject":"chinese","question":"热是什么意思？","answer":"热"},
    {"questionID":"chin_60","subject":"chinese","question":"冷是什么意思？","answer":"冷"},
    {"questionID":"chin_61","subject":"chinese","question":"高是什么意思？","answer":"高"},
    {"questionID":"chin_62","subject":"chinese","question":"低是什么意思？","answer":"低"},
    {"questionID":"chin_63","subject":"chinese","question":"新是什么意思？","answer":"新"},
    {"questionID":"chin_64","subject":"chinese","question":"旧是什么意思？","answer":"旧"},
    {"questionID":"chin_65","subject":"chinese","question":"左是什么意思？","answer":"左"},
    {"questionID":"chin_66","subject":"chinese","question":"右是什么意思？","answer":"右"},
    {"questionID":"chin_67","subject":"chinese","question":"上是什么意思？","answer":"上"},
    {"questionID":"chin_68","subject":"chinese","question":"下是什么意思？","answer":"下"},
    {"questionID":"chin_69","subject":"chinese","question":"前是什么意思？","answer":"前"},
    {"questionID":"chin_70","subject":"chinese","question":"后是什么意思？","answer":"后"},
    {"questionID":"chin_71","subject":"chinese","question":"里是什么意思？","answer":"里"},
    {"questionID":"chin_72","subject":"chinese","question":"外是什么意思？","answer":"外"},
    {"questionID":"chin_73","subject":"chinese","question":"中是什么意思？","answer":"中"},
    {"questionID":"chin_74","subject":"chinese","question":"红色是什么意思？","answer":"红色"},
    {"questionID":"chin_75","subject":"chinese","question":"黄色是什么意思？","answer":"黄色"},
    {"questionID":"chin_76","subject":"chinese","question":"蓝色是什么意思？","answer":"蓝色"},
    {"questionID":"chin_77","subject":"chinese","question":"绿色是什么意思？","answer":"绿色"},
    {"questionID":"chin_78","subject":"chinese","question":"白色是什么意思？","answer":"白色"},
    {"questionID":"chin_79","subject":"chinese","question":"黑色是什么意思？","answer":"黑色"},
    {"questionID":"chin_80","subject":"chinese","question":"狗是什么意思？","answer":"狗"},
    {"questionID":"chin_81","subject":"chinese","question":"猫是什么意思？","answer":"猫"},
    {"questionID":"chin_82","subject":"chinese","question":"鸟是什么意思？","answer":"鸟"},
    {"questionID":"chin_83","subject":"chinese","question":"鱼是什么意思？","answer":"鱼"},
    {"questionID":"chin_84","subject":"chinese","question":"马是什么意思？","answer":"马"},
    {"questionID":"chin_85","subject":"chinese","question":"牛是什么意思？","answer":"牛"},
    {"questionID":"chin_86","subject":"chinese","question":"羊是什么意思？","answer":"羊"},
    {"questionID":"chin_87","subject":"chinese","question":"猪是什么意思？","answer":"猪"},
    {"questionID":"chin_88","subject":"chinese","question":"鸡是什么意思？","answer":"鸡"},
    {"questionID":"chin_89","subject":"chinese","question":"手是什么意思？","answer":"手"},
    {"questionID":"chin_90","subject":"chinese","question":"脚是什么意思？","answer":"脚"},
    {"questionID":"chin_91","subject":"chinese","question":"头是什么意思？","answer":"头"},
    {"questionID":"chin_92","subject":"chinese","question":"眼睛是什么意思？","answer":"眼睛"},
    {"questionID":"chin_93","subject":"chinese","question":"耳朵是什么意思？","answer":"耳朵"},
    {"questionID":"chin_94","subject":"chinese","question":"嘴是什么意思？","answer":"嘴"},
    {"questionID":"chin_95","subject":"chinese","question":"鼻子是什么意思？","answer":"鼻子"},
    {"questionID":"chin_96","subject":"chinese","question":"百是什么意思？","answer":"百"},
    {"questionID":"chin_97","subject":"chinese","question":"千是什么意思？","answer":"千"},
    {"questionID":"chin_98","subject":"chinese","question":"万是什么意思？","answer":"万"},
    {"questionID":"chin_99","subject":"chinese","question":"中国货币？","answer":"人民币"},
    {"questionID":"chin_100","subject":"chinese","question":"中国国旗颜色？","answer":"红色"},
    {"questionID":"chin_101","subject":"chinese","question":"中国最大河流？","answer":"长江"},
    {"questionID":"chin_102","subject":"chinese","question":"中国第二大河流？","answer":"黄河"},
    {"questionID":"chin_103","subject":"chinese","question":"中国最高山峰？","answer":"珠穆朗玛峰"},
    {"questionID":"chin_104","subject":"chinese","question":"中国国花？","answer":"牡丹"},
    {"questionID":"chin_105","subject":"chinese","question":"中国国宝？","answer":"大熊猫"},
    {"questionID":"chin_106","subject":"chinese","question":"中国主要语言？","answer":"普通话"},
    {"questionID":"chin_107","subject":"chinese","question":"中国第一大城市？","answer":"上海"},
    {"questionID":"chin_108","subject":"chinese","question":"天是什么意思？","answer":"天"},
    {"questionID":"chin_109","subject":"chinese","question":"地是什么意思？","answer":"地"},
    {"questionID":"chin_110","subject":"chinese","question":"山是什么意思？","answer":"山"},
    {"questionID":"chin_111","subject":"chinese","question":"河是什么意思？","answer":"河"},
    {"questionID":"chin_112","subject":"chinese","question":"树是什么意思？","answer":"树"},
    {"questionID":"chin_113","subject":"chinese","question":"花是什么意思？","answer":"花"},
    {"questionID":"chin_114","subject":"chinese","question":"草是什么意思？","answer":"草"},
    {"questionID":"chin_115","subject":"chinese","question":"风是什么意思？","answer":"风"},
    {"questionID":"chin_116","subject":"chinese","question":"雨是什么意思？","answer":"雨"},
    {"questionID":"chin_117","subject":"chinese","question":"雪是什么意思？","answer":"雪"},
    {"questionID":"chin_118","subject":"chinese","question":"云是什么意思？","answer":"云"},
    {"questionID":"chin_119","subject":"chinese","question":"太阳是什么意思？","answer":"太阳"},
    {"questionID":"chin_120","subject":"chinese","question":"月亮是什么意思？","answer":"月亮"},
    {"questionID":"chin_121","subject":"chinese","question":"星星是什么意思？","answer":"星星"},
    {"questionID":"chin_122","subject":"chinese","question":"车是什么意思？","answer":"车"},
    {"questionID":"chin_123","subject":"chinese","question":"门是什么意思？","answer":"门"},
    {"questionID":"chin_124","subject":"chinese","question":"窗是什么意思？","answer":"窗"},
    {"questionID":"chin_125","subject":"chinese","question":"路是什么意思？","answer":"路"}
    ]
with open("knowledge_base.jsonl", "w", encoding="utf-8") as f:
    for entry in knowledge_base:
        f.write(json.dumps(entry, ensure_ascii=False) + "\n")

In [3]:
# STEP 3: Model & generation config
MODEL_NAME = "Qwen/Qwen3-1.7B"
CACHE_DIR = os.path.expanduser("C:/Users/ADITYA/Downloads/LLM_MAX/Qwen/models")
TEMPERATURE = 0.01
TOP_K = 5
TOP_P = 0.2
MAX_NEW_TOKENS = 60
BATCH_SIZE = 5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"



In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM,BitsAndBytesConfig
from huggingface_hub import snapshot_download

model_path = snapshot_download(
    repo_id=MODEL_NAME,
    cache_dir=CACHE_DIR,
    ignore_patterns=["*.gguf", "*.bin", "*.msgpack"]
)

# Set environment vars BEFORE imports/loads
os.environ["TRANSFORMERS_OFFLINE"] = "1"
os.environ["HF_HUB_OFFLINE"] = "1"
os.environ["HF_DATASETS_OFFLINE"] = "1"

tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True, trust_remote_code=True, cache_dir=CACHE_DIR)
tokenizer.padding_side = 'left'

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    local_files_only=True,
    cache_dir=CACHE_DIR,
    device_map="auto" if torch.cuda.is_available() else "cpu",
    use_cache=True
)

model.eval()



Fetching 12 files: 100%|██████████| 12/12 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:05<00:00,  2.71s/it]



Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layer

In [5]:
# Dense encoder for semantic retrieval
dense_encoder = SentenceTransformer("all-MiniLM-L6-v2")  # Use MiniLM for speed
kb_texts = [entry["question"] + " " + entry["answer"] for entry in knowledge_base]
dense_vectors = dense_encoder.encode(kb_texts, convert_to_tensor=True)  # shape: (500, hidden_dim)

# BM25 for keyword sparse retrieval
kb_words = [entry["question"].split() + entry["answer"].split() for entry in knowledge_base]
bm25 = BM25Okapi(kb_words)


In [6]:
fewshot_examples = {
    "algebra": [
        {"question": "What is a polynomial?", "answer": "Sum of terms with variables."},
        {"question": "What is a quadratic equation?", "answer": "ax² + bx + c = 0"}
    ],
    "geography": [
        {"question": "What is a peninsula?", "answer": "Land with water on three sides."},
        {"question": "What is the largest continent?", "answer": "Asia"}
    ],
    "history": [
        {"question": "Who was Cleopatra?", "answer": "Egyptian queen."},
        {"question": "What was the Renaissance?", "answer": "Cultural rebirth in Europe."}
    ],
    "chinese": [
        {"question": "[translate:中国的首都是哪里？]", "answer": "[translate:北京]"},
        {"question": "[translate:中国的国花是什么？]", "answer": "[translate:牡丹]"}
    ]
}


In [7]:
def hybrid_retrieve(query, top_k=2):
    # Dense
    q_vec = dense_encoder.encode(query, convert_to_tensor=True)
    dense_scores = (dense_vectors @ q_vec).cpu().numpy()
    dense_top_ids = dense_scores.argsort()[-top_k:][::-1]
    # Sparse
    bm25_scores = bm25.get_scores(query.split())
    bm25_top_ids = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]
    # Combine, deduplicate
    combined_ids = list(dict.fromkeys(list(dense_top_ids) + list(bm25_top_ids)))
    return [knowledge_base[i] for i in combined_ids]


In [8]:
# Cell 7: Add a prompt verification/clarification step at the end
def add_stepback_context(question, context, subject):
    return (
        f"{context}\n"
        f"Before answering, step back and clarify why this response is accurate for {subject} questions."
        f"\nQuestion: {question}\nAnswer:"
    )


In [9]:
# Cell 8: Dynamic prompt engineering (context + fewshot + step-back)
def build_prompt(question, subject, context, fewshot):
    sys_prompts = {
        "algebra": "You are an algebra expert.",
        "geography": "You are a geography expert.",
        "history": "You are a history expert.",
        "chinese": "You are a Chinese language and culture expert."
    }
    shots = "\n".join([f"Q: {ex['question']}\nA: {ex['answer']}" for ex in fewshot])
    prompt = f"{sys_prompts[subject]}\n{shots}\nContext: {context}"
    return add_stepback_context(question, prompt, subject)


In [10]:
def batched_hybrid_generate(questions, tokenizer, model, batch_size=16, top_k=2):
    results = []
    for i in range(0, len(questions), batch_size):
        batch = questions[i:i+batch_size]
        prompts = []
        meta_ids = []
        for q in batch:
            ctx_entries = hybrid_retrieve(q["question"], top_k=top_k)
            prompt = build_prompt(q["question"], q["subject"], ctx_entries, fewshot_examples[q["subject"]])
            prompts.append(prompt)
            meta_ids.append(q["questionID"])
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False,
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        answers = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for idx, prompt_text in enumerate(prompts):
            answer = answers[idx][len(prompt_text):].strip()
            results.append({"questionID": meta_ids[idx], "answer": answer})
    return results


In [11]:
# Cell 10: Full pipeline loader per challenge format
def loadPipeline():
    return lambda questions: batched_generate(
        questions, knowledge_graph, tokenizer, model, fewshot_examples, batch_size=BATCH_SIZE
    )


In [12]:
import time

# Cell: Run pipeline and measure time for all questions
pipeline = loadPipeline()

test_questions = [
    {"questionID": entry["questionID"], "subject": entry["subject"], "question": entry["question"]}
    for entry in knowledge_base
]

start_time = time.time()
results = batched_hybrid_generate(test_questions, tokenizer, model, batch_size=10)
end_time = time.time()

print(f"Total time for {len(test_questions)} questions: {end_time - start_time:.2f} seconds")
print(f"Avg per question: {(end_time - start_time)/len(test_questions):.3f} seconds")

for res in results[:5]:
    answer_lines = res['answer'].split('\n')
    for line in answer_lines:
        if line.strip().startswith('Question:'):
            print(f"Q: {line.strip()[len('Question:'):].strip()}")
        elif line.strip().startswith('Answer:'):
            print(f"A: {line.strip()[len('Answer:'):].strip()}")
        elif line.strip() and not line.strip().startswith('Why') and not line.strip().startswith('Are these'):
            print(f"A: {line.strip()}")
    print()


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Total time for 499 questions: 1719.66 seconds
Avg per question: 3.446 seconds
A: 4
Q: What is 2 + 8?
A: 10
Q: What is 1 + 1?
A: 2
A: The answers are correct. The questions are all simple addition problems, and the answers

A: 8
Q: What is 5 + 5?
A: 10
Q: What is 15 ÷ 3?
A: 5
A: Okay, let's see. The user is asking if the answers provided

A: 6
A: A: Because it's a basic arithmetic operation.
A: The answer is correct because it's a basic arithmetic operation. The question is a simple subtraction problem, and the answer is derived directly from the arithmetic rules. In algebra, basic operations like addition

A: 4
A: A: Because it's a basic arithmetic operation.
A: The answer is accurate because it's a basic arithmetic operation.
A: The answer is accurate because it's a basic arithmetic operation.
A: The answer is accurate because it's a basic arithmetic operation.
A: The answer

A: 12
A: A: Because 3 × 4 is a basic arithmetic operation, and it is a fundamental concept in algebra. In alge

In [ ]:
import os
import zipfile
from pathlib import Path

submission_code = '''"""Tech Arena 2025 - Team LLM-Max

Strategy:
- 1500+ Knowledge Base entries → Geography/History/Algebra/Chinese
- Hybrid retrieval → dense (semantic) + sparse (keyword)
- Dynamic prompt engineering → subject-specific few-shot + context + step-back verification
- Batched generation → efficiency
- Model: Qwen 3.1 1.7B → balance performance & speed
- Low temperature → accuracy
"""
'''
submission_code += ''' 
# Dense encoder for semantic retrieval
dense_encoder = SentenceTransformer("all-MiniLM-L6-v2")  # Use MiniLM for speed
kb_texts = [entry["question"] + " " + entry["answer"] for entry in knowledge_base]
dense_vectors = dense_encoder.encode(kb_texts, convert_to_tensor=True)  # shape: (500, hidden_dim)

# BM25 for keyword sparse retrieval
kb_words = [entry["question"].split() + entry["answer"].split() for entry in knowledge_base]
bm25 = BM25Okapi(kb_words)
'''

submission_code += '''
fewshot_examples = {
    "algebra": [
        {"question": "What is a polynomial?", "answer": "Sum of terms with variables."},
        {"question": "What is a quadratic equation?", "answer": "ax² + bx + c = 0"}
    ],
    "geography": [
        {"question": "What is a peninsula?", "answer": "Land with water on three sides."},
        {"question": "What is the largest continent?", "answer": "Asia"}
    ],
    "history": [
        {"question": "Who was Cleopatra?", "answer": "Egyptian queen."},
        {"question": "What was the Renaissance?", "answer": "Cultural rebirth in Europe."}
    ],
    "chinese": [
        {"question": "[translate:中国的首都是哪里？]", "answer": "[translate:北京]"},
        {"question": "[translate:中国的国花是什么？]", "answer": "[translate:牡丹]"}
    ]
}
'''

submission_code += '''
def hybrid_retrieve(query, top_k=2):
    # Dense
    q_vec = dense_encoder.encode(query, convert_to_tensor=True)
    dense_scores = (dense_vectors @ q_vec).cpu().numpy()
    dense_top_ids = dense_scores.argsort()[-top_k:][::-1]
    # Sparse
    bm25_scores = bm25.get_scores(query.split())
    bm25_top_ids = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]
    # Combine, deduplicate
    combined_ids = list(dict.fromkeys(list(dense_top_ids) + list(bm25_top_ids)))
    return [knowledge_base[i] for i in combined_ids]
'''
submission_code += '''
# Cell 7: Add a prompt verification/clarification step at the end
def add_stepback_context(question, context, subject):
    return (
        f"{context}"
        f"Before answering, step back and clarify why this response is accurate for {subject} questions."
        f" Question: {question}  Answer:"
    )

'''
submission_code += '''
# Cell 8: Dynamic prompt engineering (context + fewshot + step-back)
def build_prompt(question, subject, context, fewshot):
    sys_prompts = {
        "algebra": "You are an algebra expert.",
        "geography": "You are a geography expert.",
        "history": "You are a history expert.",
        "chinese": "You are a Chinese language and culture expert."
    }
    shots = " ".join([f"Q: {ex['question']} A : {ex['answer']}" for ex in fewshot])
    prompt = f"{sys_prompts[subject]} {shots} Context: {context}"
    return add_stepback_context(question, prompt, subject)

'''

submission_code += '''
def batched_hybrid_generate(questions, tokenizer, model, batch_size=16, top_k=2):
    results = []
    for i in range(0, len(questions), batch_size):
        batch = questions[i:i+batch_size]
        prompts = []
        meta_ids = []
        for q in batch:
            ctx_entries = hybrid_retrieve(q["question"], top_k=top_k)
            prompt = build_prompt(q["question"], q["subject"], ctx_entries, fewshot_examples[q["subject"]])
            prompts.append(prompt)
            meta_ids.append(q["questionID"])
        inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True).to(model.device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_NEW_TOKENS,
                do_sample=False,
                use_cache=True,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        answers = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        for idx, prompt_text in enumerate(prompts):
            answer = answers[idx][len(prompt_text):].strip()
            results.append({"questionID": meta_ids[idx], "answer": answer})

            
    return results
'''
submission_code += '''
def loadPipeline():
    return lambda questions: batched_generate(
        questions, knowledge_graph, tokenizer, model, fewshot_examples, batch_size=BATCH_SIZE
    )
'''

submission_code += '''
import time

# Cell: Run pipeline and measure time for all questions
pipeline = loadPipeline()

test_questions = [
    {"questionID": entry["questionID"], "subject": entry["subject"], "question": entry["question"]}
    for entry in knowledge_base
]

start_time = time.time()
results = batched_hybrid_generate(test_questions, tokenizer, model, batch_size=10)
end_time = time.time()

print(f"Total time for {len(test_questions)} questions: {end_time - start_time:.2f} seconds")
print(f"Avg per question: {(end_time - start_time)/len(test_questions):.3f} seconds")

for res in results[:5]:
    print(json.dumps(res, ensure_ascii=False, indent=2))

'''


In [ ]:
import os
import zipfile
from pathlib import Path


# Base directory and paths
base_dir = os.path.expanduser("C:/Users/ADITYA/Downloads/submission")
zip_path = os.path.join(base_dir, "inferencePipeline.zip")

# Ensure directories exist
try:
    os.makedirs(os.path.join(base_dir, "inferencePipeline"), exist_ok=True)
except Exception as e:
    print(f"Error creating directories: {e}")

load_path = os.path.join(base_dir, "inferencePipeline", "load.py")
init_path = os.path.join(base_dir, "inferencePipeline", "__init__.py")
req_file = os.path.join(base_dir,"inferencePipeline", "requirements.txt")

# Debugging: Print paths
print(f"load_path: {load_path}")
print(f"req_file: {req_file}")

# Create load.py
try:
    with open(load_path, 'w', encoding='utf-8') as f:
        f.write(submission_code)
    print("load.py created successfully.")
except Exception as e:
    print(f"Error creating load.py: {e}")

try:
    with open(init_path, 'w', encoding='utf-8') as f:
        f.write("from .load import loadPipeline")
    print("__init__.py created successfully.")
except Exception as e:
    print(f"Error creating __init__.py: {e}")

# Create requirements.txt
requirements = """
transformers
sentence-transformers
torch
rank_bm25
huggingface-hub
numpy
json
"""
try:
    with open(req_file, 'w', encoding='utf-8') as f:
        f.write(requirements)
    print("requirements.txt created successfully.")
except Exception as e:
    print(f"Error creating requirements.txt: {e}")

# Create ZIP archive
try:
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        zf.write(load_path, "load.py")
        zf.write(init_path, '__init__.py')
        zf.write(init_path, 'requirements.txt')
    print("ZIP archive created successfully.")
except Exception as e:
    print(f"Error creating ZIP archive: {e}")

# Get ZIP size
try:
    zip_size_kb = Path(zip_path).stat().st_size / 1024
    print(f"ZIP size: {zip_size_kb:.2f} KB")
except Exception as e:
    print(f"Error calculating ZIP size: {e}")

load_path: C:/Users/ADITYA/Downloads/submission\inferencePipeline\load.py
req_file: C:/Users/ADITYA/Downloads/submission\inferencePipeline\requirements.txt
load.py created successfully.
__init__.py created successfully.
requirements.txt created successfully.
ZIP archive created successfully.
ZIP size: 2.44 KB
